In [ ]:
#from sys import path
#path.append('./funciones')

Importo el dataset de manera general

In [ ]:
import pandas as pd
import numpy as np

data_location = './datasets/properatti.csv'

raw_data = pd.read_csv(data_location)

In [ ]:
data = raw_data
data.index

Dataset

#Completar con nombre de campo y tipo de dato

Porcentaje de faltantes de datos

In [ ]:
data.info()

In [ ]:
data.sample(10)

In [ ]:
from funciones import *

In [ ]:
missing_zero_values_table(data)

Nos deshacemos de la columna de indice ya que tienen los mismos valores.

In [ ]:
#data.reset_index(drop=True, inplace=True)
#data.index

### Análisis de la variable _operation_

In [ ]:
data['operation'].value_counts()

Todas las propiedades son para la venta, lo cual representa una categoría única.

In [ ]:
data['operation_category'] = data['operation'].astype('category')
data['operation_category'].dtype

### Análisis de la variable _property_type_

In [ ]:
data['property_type'].value_counts()

Observamos que tiene tiene cuatro posibles opciones por lo cual la cambiamos a categórica.

In [ ]:
data['property_type_category'] = data['property_type'].astype('category')
data['property_type_category'].dtype

### Análisis de la variable _place_name_

In [ ]:
data['place_name'].value_counts()
#data[['place_name','place_with_parent_names','country_name','state_name']].sample(10)

Haciendo un análisis de las 23 observaciones que no tienen 'place_name', observamos que las mismas pertenecen al barrio de Rincón de Mimberg, y hay observaciones que tienen las mismas características para descripciones similares. Sumado a que geográficamente las descripciones apuntan a que se encuentran en barrios cerrados ó courtries en dicha ubicación.

In [ ]:
data_place_name_null = data.loc[data['place_name'].isnull()]
data_place_name_null.head(5)

In [ ]:
data[data['place_name'].str.contains('Rincón de Milberg',na=False)].head(5)

Por este motivo imputamos a los datos faltantes con Rincón de Milberg, ajustando al mismo tiempo el campo place_with_parent_names para que sea consistente con la información del dataset.

In [ ]:
data['place_name_check'] = data['place_name']
data['place_with_parent_names_check'] = data['place_with_parent_names']

data.loc[data['place_name'].isnull(),['place_name_check','place_with_parent_names_check']] = \
    ['Rincón de Milberg','|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Rincón de Milberg|']
data_place_name_null

Comprobamos que no haya faltantes en las columnas que incluimos extra

In [ ]:
missing_zero_values_table(data)

### Análisis de la variable _country_name_

In [ ]:
data['country_name'].value_counts()

No es necesario realizar ningún tipo de ajuste en esta variable. Vemos que todas las propiedades son en Argentina.

### Análisis de la variable _state_name_

In [ ]:
data['state_name'].value_counts()

No es necesario realizar ningún tipo de ajuste en esta variable. Observamos que la provinicia de Buenos Aires ha sido dividida en regiones geográficas, a diferencia del resto de las provincias.

### Análisis de las variables de Geoposicionamiento

Corroboramos que todas las observaciones que tienen datos en lat-lon tienen tambien en lat y lon

In [ ]:
mask_lat_notnull = data['lat'].notnull()
mask_lon_notnull = data['lat'].notnull()
mask_lat_lon_notnull = data['lat-lon'].notnull()

mask_comparison_lat_lon = mask_lat_notnull & mask_lon_notnull & mask_lat_lon_notnull

print(f"Cantidad de observaciones con datos de Latitud y Longitud: {mask_lat_lon_notnull.sum()}")
print(f"Cantidad de observaciones en donde si hay datos de Latitud y Longitud también hay datos en las columnas separadas: {mask_comparison_lat_lon.sum()}")


data.loc[mask_comparison_lat_lon,['lat-lon','lat','lon']]

Corroboramos que los valores sean los mismos luego de haber hecho la división en columnas

In [ ]:
import re

pattern_lat_lon = '(?P<latitude>.+),(?P<longitude>.+)'
pattern_lat_lon_regex = re.compile(pattern_lat_lon)

resultados_lat_lon = data['lat-lon'].apply(lambda x: x if x is np.NaN else pattern_lat_lon_regex.match(x))

lat_match = resultados_lat_lon.apply(lambda x: x if x is np.NaN else x.group('latitude'))
lon_match = resultados_lat_lon.apply(lambda x: x if x is np.NaN else x.group('longitude'))


lat_match_numeric = lat_match.astype(float)
lon_match_numeric = lon_match.astype(float)


lat_check = data['lat'] - lat_match_numeric < .000001
lon_check = data['lon'] - lon_match_numeric < .000001


data_lat_lon_check = lat_check & lon_check

print(f"Cantidad de valores donde coinciden los valores de las columnas: {data_lat_lon_check.sum()}")


Vamos a revisar las 

In [ ]:
data['currency'].value_counts()

In [ ]:
data['currency'].notnull().sum()

mask_testing = (data['currency'].isnull()) \
        & ((data['price'].notnull()) \
        | (data['price_aprox_local_currency'].notnull()) \
        | (data['price_aprox_usd'].notnull()) \
        | (data['price_usd_per_m2'].notnull()) \
        | (data['price_per_m2'].notnull()))
mask_testing.sum()
data[mask_testing]

Para aquellas observaciones para las cuales únicamente tenemos el geoname_id le obtenemos la latitud y longitud correspondiente

NO por ahora.

In [ ]:
data.head(10)

In [ ]:
data.state_name.unique()

In [ ]:
data.loc[data['state_name']=='Capital Federal']['place_name'].unique()

In [ ]:
data.loc[data.currency.isna()]

In [ ]:
mask1 = (data['currency'].isna()) \
        & (data['price'].isna()) \
        & (data['price_aprox_local_currency'].isna()) \
        & (data['price_aprox_usd'].isna()) \
        & (data['price_usd_per_m2'].isna()) \
        & (data['price_per_m2'].isna())

mask2 = (data['price'].isna()) \
        & (data['price_aprox_local_currency'].isna()) \
        & (data['price_aprox_usd'].isna()) \
        & (data['price_usd_per_m2'].isna()) \
        & (data['price_per_m2'].isna())

#print(mask1.value_counts())
#print(mask2.value_counts())
data.loc[mask2,:].head(5)['description']

In [ ]:
data.iloc[67,:][{'description','title'}].description

In [ ]:
data.describe()

In [ ]:
df['description'].sample(10)

In [ ]:
df.info()

In [ ]:
import numpy as np
col_floor = df.floor

In [ ]:
df[df[]]

In [ ]:
df.iloc[121217].geonames_id

In [ ]:
df_map_test = df[~df['lat'].isna()]
df_map_test.sample(30000)

In [ ]:
df.iloc[26637]

In [ ]:
df.info()

In [ ]:
df